In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
 #  for filename in filenames:
  #      print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Import Mdules

In [ ]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import librosa
import librosa.display
from IPython.display import Audio
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
import warnings
warnings.filterwarnings('ignore')


## load the dataset

In [ ]:
paths = []
labels = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        paths.append(os.path.join(dirname, filename))
        label = filename.split('_')[-1]
        label = label.split('.')[0]
        labels.append(label.lower())
print('Dataset is loaded')

In [ ]:
paths[:5]

In [ ]:
labels[:5]

In [ ]:

df = pd.DataFrame()
df['speech'] = paths
df['label'] =labels
df.head()

In [ ]:
df['label'].value_counts()

## EXPLORATORY DATA ANALYSIS

In [ ]:
sns.countplot(x=df['label'])

In [ ]:
def waveplot(data, sr, emotion):
    plt.figure(figsize=(10,4))
    plt.title(emotion, size=20)
    librosa.display.waveshow(data, sr=sr)
    plt.show()
    
def spectrogram(data, sr, emotion):
    X = librosa.stft(data)
    Xdb = librosa.amplitude_to_db(abs(X))
    plt.figure(figsize=(11, 4))
    librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='hz')
    plt.colorbar()
    plt.title(emotion, size=20)
    plt.show()

In [ ]:
emotion = 'fear'
path = df['speech'][df['label']==emotion][0]
data, sampling_rate = librosa.load(path)
waveplot(data,sampling_rate,emotion)
spectrogram(data, sampling_rate,emotion)
Audio(path)

In [ ]:
emotion = 'angry'
path = np.array(df['speech'][df['label']==emotion])[0]
data, sampling_rate = librosa.load(path)
waveplot(data,sampling_rate,emotion)
spectrogram(data, sampling_rate,emotion)
Audio(path)

In [ ]:
emotion = 'disgust'
path = np.array(df['speech'][df['label']==emotion])[0]
data, sampling_rate = librosa.load(path)
waveplot(data,sampling_rate,emotion)
spectrogram(data, sampling_rate,emotion)
Audio(path)

In [ ]:
emotion = 'neutral'
path = np.array(df['speech'][df['label']==emotion])[0]
data, sampling_rate = librosa.load(path)
waveplot(data,sampling_rate,emotion)
spectrogram(data, sampling_rate,emotion)
Audio(path)

In [ ]:
emotion = 'sad'
path = np.array(df['speech'][df['label']==emotion])[0]
data, sampling_rate = librosa.load(path)
waveplot(data,sampling_rate,emotion)
spectrogram(data, sampling_rate,emotion)
Audio(path)

In [ ]:
emotion = 'ps'
path = np.array(df['speech'][df['label']==emotion])[0]
data, sampling_rate = librosa.load(path)
waveplot(data,sampling_rate,emotion)
spectrogram(data, sampling_rate,emotion)
Audio(path)

In [ ]:
emotion = 'happy'
path = np.array(df['speech'][df['label']==emotion])[0]
data, sampling_rate = librosa.load(path)
waveplot(data,sampling_rate,emotion)
spectrogram(data, sampling_rate,emotion)
Audio(path)

## feature extraction

In [ ]:
def extract_mfcc(filename):
    y, sr= librosa.load(filename, duration= 3, offset= 0.5)
    mfcc = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=50).T, axis=0)
    return mfcc

In [ ]:
extract_mfcc(df['speech'][0])

In [ ]:
X_mfcc= df['speech'].apply(lambda x: extract_mfcc(x))

In [ ]:
X_mfcc

In [ ]:
X = [x for x in X_mfcc]
X = np.array(X)
X.shape

In [ ]:
## input 
X= np.expand_dims(X, -1)
X.shape

In [ ]:

from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
y_sparse = enc.fit_transform(df[['label']])


In [ ]:
if isinstance(y_sparse, np.ndarray):
    y = y_sparse
else:
    y = y_sparse.toarray()

In [ ]:
y.shape

In [ ]:
from keras.models import Sequential
from keras.layers import Dense ,LSTM, Dropout

model = Sequential([LSTM(128,return_sequences=False, input_shape=(50,1)),
                   Dense(64, activation='relu'),
                   Dropout(0.2),
                   Dense(32, activation='relu'), 
                   Dropout(0.2),
                   Dense(7, activation='softmax')
                   ])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

## model training

In [ ]:
history = model.fit(X, y, validation_split=0.2, epochs=100, batch_size=512, shuffle=True)


## plot result

In [ ]:
epochs = list(range(100))
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

plt.plot(epochs, acc, label='train accuracy')
plt.plot(epochs, val_acc, label='val_accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.plot(epochs, acc, label='train loss')
plt.plot(epochs, val_acc, label='val_loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()